# CM6_Project by Marshal Pfluger
### Dr. Sajjad Mohsin

#### In this project we will be using reinforcement learning to build a model of our choice. For this project, I chose to look at the OpenAI Gym libraries and have narrowed my sights on the mountain car problem. In this challenge we have a little car that is trying to get up a slope to a target point. The car does not have enough power to do it straight up so it will need to build momentum going back and forth to get to the target zone. We will see if we can reach the target zone with our reinforcement model. Little did I know this would be the most challenging project of the semester. See the conclusion for a breakdown of what occurred.

### first we will import necessary dependencies.

In [1]:
# install dependencies needed for recording videos
!pip install gym
!apt-get install -y xvfb x11-utils
!pip install pyvirtualdisplay

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 libxtst6 libxxf86dga1 x11-xkb-utils xfonts-base
  xfonts-encodings xfonts-utils xserver-common
Suggested packages:
  mesa-utils
The following NEW packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 libxtst6 libxxf86dga1 x11-utils x11-xkb-utils xfonts-base
  xfonts-encodings xfonts-utils xserver-common xvfb
0 upgraded, 12 newly installed, 0 to remove and 10 not upgraded.
Need to get 8,045 kB of archives.
After this operation, 12.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfontenc1 amd64 1:1.1.4-1build3 [14.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxfont2 amd64 1:2.0.5-1build1 [94.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxkbfile1 amd64 1:1.1.0-1build3 [71.8 kB]
Get:4 http://archive.ubuntu.

In [2]:
# Import the libraries for operation
import numpy as np
import gym
from pyvirtualdisplay import Display
import time
from gym.wrappers.monitoring.video_recorder import VideoRecorder
from base64 import b64encode
from IPython.display import HTML

#### Now we will create the environment using the Mountain Car gymnasium

In [3]:
# Create the environment
env = gym.make("MountainCar-v0")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [4]:
# Check the environments action space
env.action_space

Discrete(3)

In [5]:
# returns an initial observation
env.reset()
for i in range(20):
  # env.action_space.sample() produces either 0 (left) or 1 (right).
  observation, reward, done, info = env.step(env.action_space.sample())
  print("step", i, observation, reward, done, info)
env.close()

step 0 [-0.45975825  0.00052801] -1.0 False {}
step 1 [-0.46070614 -0.00094788] -1.0 False {}
step 2 [-0.46212292 -0.00141678] -1.0 False {}
step 3 [-0.46499816 -0.00287524] -1.0 False {}
step 4 [-0.46931064 -0.00431248] -1.0 False {}
step 5 [-0.47402847 -0.00471784] -1.0 False {}
step 6 [-0.47911674 -0.00508825] -1.0 False {}
step 7 [-0.4835376  -0.00442088] -1.0 False {}
step 8 [-0.48725823 -0.00372062] -1.0 False {}
step 9 [-0.49225086 -0.00499263] -1.0 False {}
step 10 [-0.49647823 -0.00422739] -1.0 False {}
step 11 [-0.50190884 -0.00543057] -1.0 False {}
step 12 [-0.506502   -0.00459313] -1.0 False {}
step 13 [-0.51022327 -0.0037213 ] -1.0 False {}
step 14 [-0.5140448  -0.00382159] -1.0 False {}
step 15 [-0.5169381  -0.00289324] -1.0 False {}
step 16 [-0.5208813  -0.00394319] -1.0 False {}
step 17 [-0.52384484 -0.00296357] -1.0 False {}
step 18 [-0.5278066  -0.00396172] -1.0 False {}
step 19 [-0.53173673 -0.00393017] -1.0 False {}


In [6]:
# Initalize the display size for output.
display = Display(visible=False, size=(1400, 900))
_ = display.start()

#### Now we will train our model using q-learning. we will use 5000 episodes. I had to tweak the hyperparameters a lot to get it where i wanted it. when reaching the end of training, the notebook will capture frames of the gym and save them to visualize what is occuring.

In [7]:

# Function to discretize the state
def discretize_state(state, num_buckets):
    state_bounds = [env.observation_space.low, env.observation_space.high]
    discretized_state = [np.linspace(start, end, num_buckets) for start, end in zip(state_bounds[0], state_bounds[1])]
    discretized_state_indices = [np.digitize(s, bins) for s, bins in zip(state, discretized_state)]
    return sum([(index - 1) * (num_buckets**i) for i, index in enumerate(discretized_state_indices)])

# Q-learning parameters
learning_rate = 0.2
discount_factor = .99
exploration_prob = 1.2
exploration_decay = 0.995
min_exploration_prob = 0.1

# Initialize Q-table
num_buckets = 40  # Adjust this parameter based on your requirements
action_space_size = env.action_space.n
state_space_size = num_buckets ** env.observation_space.shape[0]
q_table = np.zeros((state_space_size, action_space_size))

# Video recording
video_path = "training_video.mp4"
video_recorder = VideoRecorder(env, video_path)

# Training loop
num_episodes = 5000

for episode in range(num_episodes):
    state = env.reset()
    state = discretize_state(state, num_buckets)

    total_reward = 0

    while True:
        # Choose action using epsilon-greedy strategy
        if np.random.rand() < exploration_prob:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state])

        # Take the chosen action
        next_state, reward, done, _ = env.step(action)
        next_state = discretize_state(next_state, num_buckets)

        # Update Q-value using the Q-learning update rule
        q_table[state][action] += learning_rate * (
            reward
            + discount_factor * np.max(q_table[next_state])
            - q_table[state][action]
        )
        state = next_state
        total_reward += reward

        # Render the environment and capture the frame for the video
        if episode > 4995:
          env.render()
          video_recorder.capture_frame()
        if done:
            break

    # Decay exploration probability
    exploration_prob = max(
        min_exploration_prob, exploration_prob * exploration_decay
    )
    print(f"Episode {episode + 1}/{num_episodes}, Total Reward: {total_reward}")

# Close the video recorder and environment
video_recorder.close()
env.close()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment MountainCar-v0 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:101: DeprecationWarning: WARN: <class 'gym.wrappers.monitoring.video_recorder.VideoRecorder'> is marked as deprecated and will be removed in the future.
  logger.deprecation(


Episode 1/5000, Total Reward: -200.0
Episode 2/5000, Total Reward: -200.0
Episode 3/5000, Total Reward: -200.0
Episode 4/5000, Total Reward: -200.0
Episode 5/5000, Total Reward: -200.0
Episode 6/5000, Total Reward: -200.0
Episode 7/5000, Total Reward: -200.0
Episode 8/5000, Total Reward: -200.0
Episode 9/5000, Total Reward: -200.0
Episode 10/5000, Total Reward: -200.0
Episode 11/5000, Total Reward: -200.0
Episode 12/5000, Total Reward: -200.0
Episode 13/5000, Total Reward: -200.0
Episode 14/5000, Total Reward: -200.0
Episode 15/5000, Total Reward: -200.0
Episode 16/5000, Total Reward: -200.0
Episode 17/5000, Total Reward: -200.0
Episode 18/5000, Total Reward: -200.0
Episode 19/5000, Total Reward: -200.0
Episode 20/5000, Total Reward: -200.0
Episode 21/5000, Total Reward: -200.0
Episode 22/5000, Total Reward: -200.0
Episode 23/5000, Total Reward: -200.0
Episode 24/5000, Total Reward: -200.0
Episode 25/5000, Total Reward: -200.0
Episode 26/5000, Total Reward: -200.0
Episode 27/5000, Tota

/usr/local/lib/python3.10/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward

Episode 4997/5000, Total Reward: -189.0
Episode 4998/5000, Total Reward: -191.0
Episode 4999/5000, Total Reward: -189.0
Episode 5000/5000, Total Reward: -184.0


#### Now we will display the output of the training.

In [8]:

def render_mp4(videopath: str) -> str:
  """
  Gets a string containing a b4-encoded version of the MP4 video
  at the specified path.
  """
  mp4 = open(videopath, 'rb').read()
  base64_encoded_mp4 = b64encode(mp4).decode()
  return f'<video width=700 controls><source src="data:video/mp4;' \
         f'base64,{base64_encoded_mp4}" type="video/mp4"></video>'

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:

html = render_mp4(video_path)
HTML(html)

## Conclusion

#### This has been one of the toughest projects that we have done so far. part of this may have been self imposed as I really wanted to see the animation of the output. Unfortunately this can be very difficult in Colab as there is not a designated output window for the notebook like there is in jupyter notebook on the local machine. It was very very difficult to get the training and animation to agree with each other and work. It seemed like every time I tried to implement a different learning method, the video output would break. I spent hours with nothing working at all. It took a lot of patience and perserverance to get this project done. I pulled inspiration from dozens of sites and visited ~70 webpages to get the notebook working. I had to tune the hyperparameters a lot. In the end I am very proud of the notebook even though it is not the longest or prettiest. Please let me know if you have any feedback for me on this project.